### Step 1 - Decide in which city the company should be based

Decision should be based on: 
1. Number of Design Companies
2. Number of 


#### Libraries

In [6]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
import pandas as pd
import numpy as np
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import time
import requests


In [7]:
db = client.get_database('Ironhack')

In [8]:
c = db.get_collection('companies_collection')
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies_collection')

Designers (23%) and Developers (17%) compose 40% of the workforce. 

The company has a higher chance to recruit and retain good talent in those areas if they are located in a place that is either:
1. near design companies <br> 
OR
2. near successful tech startups that have raised more tham 1M 

In [9]:
# FIND DESIGN COMPANIES:
query_design = {"tag_list" : {'$regex':'.*design.*'}}


#FIND TECH STARTUPS:
tech_1 = {'total_money_raised':{'$regex':'M'}}
tech_2 = {'founded_year': {'$lte': 2015}}
tech_3 = {"category_code" : {'$regex':'tech'}}
query_tech = {'$and': [
    tech_1, tech_2, tech_3
]}
projection = {'_id': 0, 'name': 1, 'total_money_raised' : 1, 'offices.country_code' : 1, 'offices.city' : 1, 'offices.latitude': 1, 'offices.longitude': 1}

query_all = {'$or': [query_design, query_tech]}

companies_filtered = list(c.find(query_all, projection))
len(companies_filtered)
#res


956

In [10]:
design_companies = list(c.find(query_design, projection))

In [11]:
tech_companies = list(c.find(query_tech, projection))

In [12]:
companies_filtered

[{'name': 'Digg',
  'total_money_raised': '$45M',
  'offices': [{'city': 'San Francisco',
    'country_code': 'USA',
    'latitude': 37.764726,
    'longitude': -122.394523}]},
 {'name': 'Adobe Systems',
  'total_money_raised': '$0',
  'offices': [{'city': 'San Jose',
    'country_code': 'USA',
    'latitude': 37.320052,
    'longitude': -121.877636}]},
 {'name': 'Negonation',
  'total_money_raised': '$0',
  'offices': [{'city': '',
    'country_code': 'ESP',
    'latitude': 40.463667,
    'longitude': -3.74922}]},
 {'name': 'GENWI',
  'total_money_raised': '$7.1M',
  'offices': [{'city': 'Los Altos',
    'country_code': 'USA',
    'latitude': 33.8171,
    'longitude': -111.9035}]},
 {'name': 'Aviary',
  'total_money_raised': '$19M',
  'offices': [{'city': 'NY',
    'country_code': 'USA',
    'latitude': None,
    'longitude': None}]},
 {'name': 'BrightQube',
  'total_money_raised': '$850k',
  'offices': [{'city': 'Carlsbad',
    'country_code': 'USA',
    'latitude': 33.09568,
    'lo

In [13]:
lat = companies_filtered[0]['offices'][0]['latitude']
lon = companies_filtered[0]['offices'][0]['longitude']
country = companies_filtered[0]['offices'][0]['country_code']

lat, lon, country


(37.764726, -122.394523, 'USA')

In [14]:
def create_df(companies_filtered):
    data = {'company': [i['name'] for i in companies_filtered],
            'city': [i['offices'][0].get('city', np.nan) if i.get('offices') else np.nan for i in companies_filtered],
            'latitude': [i['offices'][0].get('latitude', np.nan) if i.get('offices') else np.nan for i in companies_filtered],
            'longitude': [i['offices'][0].get('longitude', np.nan) if i.get('offices') else np.nan for i in companies_filtered]
           }
    return pd.DataFrame.from_dict(data, orient='columns')

In [15]:
df_all = create_df(companies_filtered)

In [16]:
df_all

,company,city,latitude,longitude
0,Digg,San Francisco,37.764726,-122.394523
1,Adobe Systems,San Jose,37.320052,-121.877636
2,Negonation,,40.463667,-3.749220
3,GENWI,Los Altos,33.817100,-111.903500
4,Aviary,NY,NaN,NaN
...,...,...,...,...
951,Advanced Power Projects,Fremont,37.525246,-121.917842
952,ITERNOVA,Teruel,40.340133,-1.105961
953,Atlantic BT,Raleigh,35.896906,-78.680678
954,Treehouse,Orlando,NaN,NaN


In [17]:
df_all.isna().sum()

company        0
city         105
latitude     350
longitude    350
dtype: int64

In [18]:
df_all.nunique()

company      924
city         459
latitude     580
longitude    580
dtype: int64

In [19]:
df_all.dropna(subset=['latitude', 'longitude'], how='any', inplace=True)

In [20]:
df_all.nunique()

company      588
city         353
latitude     580
longitude    580
dtype: int64

In [21]:
# Create DESIGN companies df and drop Nan lat and long
df_design = create_df(design_companies)
df_design.dropna(subset=['latitude', 'longitude'], how='any', inplace=True)
df_design.sample()

,company,city,latitude,longitude
56,FliteHaus Creative Agency,Oviedo,28.544123,-81.366414


In [22]:
# Create TECH companies df and drop Nan lat and long
df_tech = create_df(tech_companies)
df_tech.dropna(subset=['latitude', 'longitude'], how='any', inplace=True)
df_tech.sample()

,company,city,latitude,longitude
216,FORMA Therapeutics,Cambridge,42.35952,-71.115691


In [23]:
#Mapping results in heat map to see potential cities
world_map = Map(location = [0, 0], zoom_start = 2)
world_map

In [24]:
# SEEING DESIGN AND TECH COMPANIES ON A MAP
tech_select = folium.FeatureGroup(name=f"Tech ({df_tech.shape[0]})")
design_select = folium.FeatureGroup(name = f"Design ({df_design.shape[0]})")
HeatMap(data = df_tech[["latitude", "longitude"]]).add_to(tech_select)
HeatMap(data = df_design[["latitude", "longitude"]]).add_to(design_select)
tech_select.add_to(world_map)
design_select.add_to(world_map)
folium.LayerControl(collapsed=False, position="topleft").add_to(world_map)
world_map

### USA and UK seem to be hubs for tech and design companies. Let's investigate

In [25]:
def top_cities(df, column, n=4, columns_to_keep=['city', 'company', 'latitude', 'longitude']):
    """
    Filters a dataframe to contain only the top n most frequent values in a specified column
    Args:
        df (pandas.DataFrame): the dataframe to be filtered
        column (str): the name of the column to count and filter
        n (int): the number of top values to keep (default is 3)
        columns_to_keep (list): the list of columns to keep in the filtered dataframe (default is ['city', 'country', 'name', 'latitude', 'longitude'])
    Returns:
        The filtered dataframe
    """
    value_counts = df[column].value_counts()
    top_values = value_counts[:n].index.tolist()
    return df[df[column].isin(top_values)][columns_to_keep]

In [26]:
top_cities = top_cities(df_all, 'city')
top_cities

,city,company,latitude,longitude
0,San Francisco,Digg,37.764726,-122.394523
2,,Negonation,40.463667,-3.749220
8,San Francisco,The Climate Corporation,37.782809,-122.394354
10,New York,Gilt Groupe,40.747270,-73.980064
17,London,cScape,51.523676,-0.107843
...,...,...,...,...
882,New York,Palyon Medical,40.756054,-73.986951
895,,Cyber-Rain,34.186206,-118.503910
899,,Delphi,42.603532,-83.161340
910,,Clean Mobile,48.055441,11.607509


### Looks like some empty cells in city were not dropped in cleaning 👀

In [27]:
# Checking the value and value type for row 0 and column 'city' to understand issue
index = 2
column = 'city'
value = df_all.iloc[index][column]
value_type = type(value)
print(f'Value at index {index} and column {column}: {value} ({value_type})')

Value at index 2 and column city:  (<class 'str'>)


In [100]:
# Work around: Drop rows where the value in column 'city' is an empty string
mask = top_cities['city'] != ''
top_cities = top_cities[mask]
top_cities.head()

,city,company,latitude,longitude
0,San Francisco,Digg,37.764726,-122.394523
8,San Francisco,The Climate Corporation,37.782809,-122.394354
10,New York,Gilt Groupe,40.747270,-73.980064
17,London,cScape,51.523676,-0.107843
19,London,ProofHQ,51.532048,-0.011154


In [101]:
def count_cities(column):
    """
    Args:
        column: the column for which you want the value counts 
    Returns:
        The value counts of the column 
    """
    value_counts = column.value_counts()
    index = value_counts.index.astype(str)
    return value_counts

In [102]:
count_cities(top_cities['city'])


New York         17
San Francisco    16
London           12
Name: city, dtype: int64

### The selected cities are `San Francisco`, `New York` and `London` 🏆

In [104]:
top_cities.to_csv("data/top_cities.csv", index=False)



#### Make citi maps:
1. Get Coordinates
2. Create Map
3. Map Companies in each city

In [105]:
def get_city_map(city, zoom=15):
    '''
    Args: 
    City: the city you want to use to create your map. The city needs to be in the below dict cities
    Zoom: how close you want the map to be, defatul being 10
    Returns: the map, centralized in the city with the zoom of choice
    '''
    cities = {
        'New York': (40.7128, -74.0060),
        'San Francisco': (37.7749, -122.4194),
        'London': (51.5074, -0.1278),
    }

    city_coordinates = {'type': 'Point', 'coordinates': cities[city]}
    return folium.Map(location=city_coordinates['coordinates'], zoom_start=zoom)

In [106]:
ny_map = get_city_map('New York')
ny_map

In [34]:
sf_map = get_city_map('San Francisco')
sf_map

In [35]:
ld_map = get_city_map('London')
ld_map

In [107]:
def map_markers(df,city,map_):
    """
    Args:
        df (dataframe): the dataframe from where we get the coordinates of each company  
        city (string): the city where we will map the companies from the df
        map (folium.Map): the map where the data wil be placed.
    Returns:
        the map with all markers
    """
    for i, row in df[df.city == city].iterrows():
        company = {
            "location":[row["latitude"], row["longitude"]],
            "tooltip" : row["company"]
        }
        
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "building",
                    icon_color = "white"
        )
            
        Marker(**company,icon = icon ).add_to(map_)
    return(map_)

In [37]:
ny_markers = map_markers(top_cities,'New York', ny_map)
ny_markers

In [108]:
ld_markers = map_markers(top_cities,'London', ld_map)
ld_markers

In [39]:
sf_markers = map_markers(top_cities,'San Francisco', sf_map)
sf_markers

## FOURSQUARE API

### Step 2 - Finding key places with FOURSQUARE API

In [109]:
## ADD SRC FILES FROM GET_COORDINATES AND FOURPSQUARE API

import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas
from getpass import getpass
load_dotenv()

True

In [41]:
token = getpass()

In [110]:
def get_coords(city):
    '''
    Args: 
    City: the city you want to use to create your map. The city needs to be in the below dict cities
    Zoom: how close you want the map to be, defatul being 10
    Returns: the map, centralized in the city with the zoom of choice
    '''
    cities = {
        'New York': [40.7128, -74.0060],
        'San Francisco': [37.7749, -122.4194],
        'London': [51.5074, -0.1278],
        
    }

    return cities[city]

In [111]:
ny_coords = get_coords('New York')
sf_coords = get_coords('San Francisco')
ld_coords = get_coords('London')
ny_coords

[40.7128, -74.006]

> `API Calls to find key places based on location AND category OR query`

In [114]:
def find_places_fsq(city_coords, category=None, query=None):
    """
    Makes calls to the foursquare API
    Args:
        city (point): coordinates of the city in type point where the request is to be made 
        query (string): the site to be searched 
        category (string): the category of the site to be searched 
    Returns:
        The request as a list
    """
    url = "https://api.foursquare.com/v3/places/search?"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    params = {
        "ll": f"{city_coords[0]},{city_coords[1]}",
        "limit": 50
    }
    
    if query and category:
        url += f'query={query}&categories={category}'
        params["query"] = query
        params["category"] = category
    elif query:
        url += f'query={query}'
        params["query"] = query
    elif category:
        url += f'categories={category}'
        params["category"] = category
        
    return requests.get(url=url, headers=headers, params=params).json()

### Find Starbucks 

In [115]:
ny_starbucks = find_places_fsq(ny_coords,query='starbucks')
sf_starbucks = find_places_fsq(sf_coords,query='starbucks')
ld_starbucks = find_places_fsq(ld_coords,query='starbucks')


### Find Schools

In [116]:
cat_code_schools = 12057 #code for Primary and Secondary School

In [117]:
ny_schools = find_places_fsq(ny_coords,category=cat_code_schools)
sf_schools = find_places_fsq(sf_coords,category=cat_code_schools)
ld_schools = find_places_fsq(ld_coords,category=cat_code_schools)

### Find Clubs

In [118]:
cat_code_clubs = 10032 #code for Primary and Secondary School 

In [119]:
ny_clubs = find_places_fsq(ny_coords,category=cat_code_clubs)
sf_clubs = find_places_fsq(sf_coords,category=cat_code_clubs)
ld_clubs = find_places_fsq(ld_coords,category=cat_code_clubs)

### Find Vegan Restaurants

In [120]:
cat_code_vegan = 13377 #Vegan and Veg restaurants

In [121]:
ny_vegan = find_places_fsq(ny_coords,category=cat_code_vegan)
sf_vegan = find_places_fsq(sf_coords,category=cat_code_vegan)
ld_vegan = find_places_fsq(ld_coords,category=cat_code_vegan)

### Find Basketball court

In [122]:
cat_code_basket = 18008

In [123]:
ny_basket = find_places_fsq(ny_coords,category=cat_code_basket)
sf_basket = find_places_fsq(sf_coords,category=cat_code_basket)
ld_basket = find_places_fsq(ld_coords,category=cat_code_basket)

In [124]:
ny_basket

{'results': [{'fsq_id': '4ba505c3f964a520c6d138e3',
   'categories': [{'id': 18008,
     'name': 'Basketball Court',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/basketballcourt_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 3121,
   'geocodes': {'main': {'latitude': 40.732345, 'longitude': -73.979407}},
   'link': '/v3/places/4ba505c3f964a520c6d138e3',
   'location': {'address': '300 1st Ave',
    'census_block': '360610044004000',
    'country': 'US',
    'cross_street': '18th Street',
    'dma': 'New York',
    'formatted_address': '300 1st Ave (18th Street), New York, NY 10009',
    'locality': 'New York',
    'postcode': '10009',
    'region': 'NY'},
   'name': 'Stuyvesant Town Basketball Courts',
   'related_places': {'parent': {'fsq_id': '4a81f0bef964a5200cf81fe3',
     'name': 'StuyTown Apartments'}},
   'timezone': 'America/New_York'},
  {'fsq_id': '4b5808a6f964a520414928e3',
   'categories': [{'id': 18008,
     'name': 'Basketb

### Find Dog Parlour

In [125]:
cat_code_pet = 11134 #Pet Gromming Service 

In [126]:
ny_pet = find_places_fsq(ny_coords,category=cat_code_pet)
sf_pet = find_places_fsq(sf_coords,category=cat_code_pet)
ld_pet = find_places_fsq(ld_coords,category=cat_code_pet)

In [127]:
ny_pet['results']

[{'fsq_id': '570eb080498e636a8f89ed64',
  'categories': [{'id': 11134,
    'name': 'Pet Grooming Service',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
     'suffix': '.png'}},
   {'id': 17110,
    'name': 'Pet Supplies Store',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
     'suffix': '.png'}}],
  'chains': [],
  'distance': 3072,
  'geocodes': {'main': {'latitude': 40.740583, 'longitude': -74.003878},
   'roof': {'latitude': 40.740583, 'longitude': -74.003878}},
  'link': '/v3/places/570eb080498e636a8f89ed64',
  'location': {'address': '327 W 14th St',
   'census_block': '360610083002002',
   'country': 'US',
   'cross_street': 'btwn 8th & 9th Ave',
   'dma': 'New York',
   'formatted_address': '327 W 14th St (btwn 8th & 9th Ave), New York, NY 10014',
   'locality': 'New York',
   'postcode': '10014',
   'region': 'NY'},
  'name': 'BeastyFeast',
  'related_places': {},
  'timezone': 'America/New_York'},
 {'fsq

In [128]:
coords = ny_pet['results'][0]["geocodes"]["main"]
coords

{'latitude': 40.740583, 'longitude': -74.003878}

In [129]:
lat, lon = coords["latitude"], coords["longitude"]

lat, lon

(40.740583, -74.003878)

### Transform the query_list of each category and location into DataFrame

In [156]:
# Function that takes the dict_ (results form four square queries) and returns a clearner dict_ with only relevant information
# name_coordinates(VARIABLE_NAME["results"]) -> variable_name is the name of the var where you saved the result of the fsq query above

def name_coordinates (dict_):
    '''
    Function that takes the dict_ (results form four square queries) and returns a clearner dict_ with only relevant information
    THIS FUNCTION WILL BE CALLED IN THE DICT_T0_DF FUNCTION.
    '''    
    processed_dict = {"name": dict_["name"],
                      #"location": {"type": "Point", "coordinates": [dict_["geocodes"]["main"]["latitude"], dict_["geocodes"]["main"]["longitude"]]}
                      'latitude': dict_["geocodes"]["main"]["latitude"],
                      'longitude': dict_["geocodes"]["main"]["longitude"]
                        }
    return processed_dict

In [157]:
def dict_to_df(query_list, category=None):
    new_list = []
    for i in query_list:
        new_list.append(name_coordinates(i))
        df = pd.DataFrame(new_list)
        if category:
            df['category'] = category
        else:
            np.nan
        #df.to_json(f"json/{query_list}.json", orient="records")
    return df

#### startbucks dfs

In [158]:
ny_starbucks_df = dict_to_df(ny_starbucks['results'], category='starbucks')
sf_starbucks_df = dict_to_df(sf_starbucks['results'], category='starbucks')
ld_starbucks_df = dict_to_df(ld_starbucks['results'], category='starbucks')
ny_starbucks_df.head()

,name,latitude,longitude,category
0,Starbucks,40.714207,-74.015710,starbucks
1,Starbucks,40.711052,-74.004848,starbucks
2,Starbucks,40.705271,-74.011165,starbucks
3,Starbucks,40.718209,-74.007401,starbucks
4,Starbucks,40.711646,-74.006792,starbucks


#### schools dfs

In [159]:
ny_schools_df = dict_to_df(ny_schools['results'], category='school')
sf_schools_df = dict_to_df(sf_schools['results'], category='school')
ld_schools_df = dict_to_df(ld_schools['results'], category='school')


#### clubs dfs

In [160]:
ny_clubs_df = dict_to_df(ny_clubs['results'], category='club')
sf_clubs_df = dict_to_df(sf_clubs['results'], category='club')
ld_clubs_df = dict_to_df(ld_clubs['results'], category='club')

#### vegan dfs

In [161]:
ny_vegan_df = dict_to_df(ny_vegan['results'], category='vegan-restaurant')
sf_vegan_df = dict_to_df(sf_vegan['results'], category='vegan-restaurant')
ld_vegan_df = dict_to_df(ld_vegan['results'], category='vegan-restaurant')

#### basketball dfs

In [162]:
ny_basket_df = dict_to_df(ny_basket['results'], category='basketball-court')
sf_basket_df = dict_to_df(sf_basket['results'], category='basketball-court')
ld_basket_df = dict_to_df(ld_basket['results'], category='basketball-court')

#### pets dfs

In [163]:
ny_pet_df = dict_to_df(ny_pet['results'], category='basketball-court')
sf_pet_df = dict_to_df(sf_pet['results'], category='basketball-court')
ld_pet_df = dict_to_df(ld_pet['results'], category='basketball-court')
ld_pet_df.head()

,name,latitude,longitude,category
0,GAIL's Bakery Muswell Hill,51.590185,-0.146090,basketball-court
1,Doggy Boutique & Grooming,51.397455,-0.186146,basketball-court
2,Smart Dogs,51.482628,-0.212610,basketball-court
3,Downtown Woof,51.447689,-0.148501,basketball-court
4,Love My Hound,51.408490,-0.039107,basketball-court


### Export dfs to json to create a company_preferences collection on Mongo_db 

In [164]:

ny_starbucks_df.to_json("json/ny_starbucks.json", orient="records")
sf_starbucks_df.to_json("json/sf_starbucks.json", orient="records")
ld_starbucks_df.to_json("json/ld_starbucks.json", orient="records")

ny_schools_df.to_json("json/ny_schools.json", orient="records")
sf_schools_df.to_json("json/sf_schools.json", orient="records")
ld_schools_df.to_json("json/ld_schools.json", orient="records")

ny_clubs_df.to_json("json/ny_clubs.json", orient="records")
sf_clubs_df.to_json("json/sf_clubs.json", orient="records")
ld_clubs_df.to_json("json/ld_clubs.json", orient="records")

ny_vegan_df.to_json("json/ny_vegan.json", orient="records")
sf_vegan_df.to_json("json/sf_vegan.json", orient="records")
ld_vegan_df.to_json("json/ld_vegan.json", orient="records")

ny_basket_df.to_json("json/ny_basket.json", orient="records")
sf_basket_df.to_json("json/sf_basket.json", orient="records")
ld_basket_df.to_json("json/ld_basket.json", orient="records")

ny_pet_df.to_json("json/ny_pet.json", orient="records")
sf_pet_df.to_json("json/sf_pet.json", orient="records")
ld_pet_df.to_json("json/ld_pet.json", orient="records")



### Concat Dfs per city

In [165]:
ny_all = pd.concat([ny_pet_df,ny_basket_df,ny_vegan_df,ny_clubs_df,ny_schools_df, ny_starbucks_df], axis=0)
ny_all.head()

,name,latitude,longitude,category
0,BeastyFeast,40.740583,-74.003878,basketball-court
1,The Salty Paw,40.707577,-74.001743,basketball-court
2,Biscuits & Bath,40.736371,-73.996113,basketball-court
3,Eva's Play Pups,40.722154,-73.959235,basketball-court
4,Dog Room Club,40.785382,-73.951538,basketball-court


In [166]:
sf_all = pd.concat([sf_pet_df,sf_basket_df,sf_vegan_df,sf_clubs_df,sf_schools_df,sf_starbucks_df], axis=0)
sf_all.head()

,name,latitude,longitude,category
0,Pet Food Express,37.787570,-122.446339,basketball-court
1,Pawtrero Hill BathHouse & Feed,37.783820,-122.389322,basketball-court
2,Wag Hotels San Francisco,37.768475,-122.414103,basketball-court
3,Wags Pet Wash & Boutique,37.793742,-122.421187,basketball-court
4,The Dog Barber,37.770428,-122.424897,basketball-court


In [167]:
ld_all = pd.concat([ld_pet_df,ld_basket_df,ld_vegan_df,ld_clubs_df,ld_schools_df,ld_starbucks_df], axis=0)
ld_all.head()

,name,latitude,longitude,category
0,GAIL's Bakery Muswell Hill,51.590185,-0.146090,basketball-court
1,Doggy Boutique & Grooming,51.397455,-0.186146,basketball-court
2,Smart Dogs,51.482628,-0.212610,basketball-court
3,Downtown Woof,51.447689,-0.148501,basketball-court
4,Love My Hound,51.408490,-0.039107,basketball-court


In [180]:
team_preferences_df = pd.concat([ny_all, sf_all, ld_all], axis=0)
team_preferences_df.head()

,name,latitude,longitude,category
0,BeastyFeast,40.740583,-74.003878,basketball-court
1,The Salty Paw,40.707577,-74.001743,basketball-court
2,Biscuits & Bath,40.736371,-73.996113,basketball-court
3,Eva's Play Pups,40.722154,-73.959235,basketball-court
4,Dog Room Club,40.785382,-73.951538,basketball-court


In [181]:
def cleanse(df): 
    """
    Double clean to Ensure that the data is clean and unique (no duplicates)
    Args:
        df (df): the dataframe that wants to be cleaned
    Returns:
        The dataframe cleaned
    """
    df = df.dropna()
    df.reset_index(drop=True,inplace=True)
    df.drop_duplicates(inplace=True)
    return df
cleanse(team_preferences_df)

,name,latitude,longitude,category
0,BeastyFeast,40.740583,-74.003878,basketball-court
1,The Salty Paw,40.707577,-74.001743,basketball-court
2,Biscuits & Bath,40.736371,-73.996113,basketball-court
3,Eva's Play Pups,40.722154,-73.959235,basketball-court
4,Dog Room Club,40.785382,-73.951538,basketball-court
...,...,...,...,...
848,Starbucks,51.508128,-0.101911,starbucks
849,Starbucks,51.530942,-0.122104,starbucks
850,Starbucks,51.516394,-0.103293,starbucks
851,Starbucks,51.514326,-0.152723,starbucks


### Export Top_Cities as json:
1. Add geometry column with Point coordinates
2. Ensure data is clean 
3. export to json

In [176]:
top_cities.head()

,city,company,latitude,longitude,geometry
0,San Francisco,Digg,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [37.764726, -..."
8,San Francisco,The Climate Corporation,37.782809,-122.394354,"{'type': 'Point', 'coordinates': [37.782809, -..."
10,New York,Gilt Groupe,40.747270,-73.980064,"{'type': 'Point', 'coordinates': [40.74727, -7..."
17,London,cScape,51.523676,-0.107843,"{'type': 'Point', 'coordinates': [51.523676, -..."
19,London,ProofHQ,51.532048,-0.011154,"{'type': 'Point', 'coordinates': [51.5320478, ..."


In [182]:
def type_point_2(df):
    """
    Creates a new column to add to the dataframe with the coordinates as a point
    Args:
        df (df): the dataframe where the new column will be created 
    Returns:
        The new column
    """
    lat = list(df.longitude) #changing order of lat and lon to fit library requirements
    long = list(df.latitude)
    coordinates = []
    for lt,lg in zip(lat,long):
        coord = [lg,lt]
        coordinates.append({"type": "Point",  "coordinates": coord})
    return coordinates

coordinates = type_point_2(top_cities)
top_cities['geometry'] = coordinates 

top_cities['geometry'].sample()


20    {'type': 'Point', 'coordinates': [37.7955307, ...
Name: geometry, dtype: object

In [191]:
team_pref_coords = type_point_2(team_preferences_df)

team_preferences_df['geometry'] = team_pref_coords 

team_preferences_df

,name,latitude,longitude,category,geometry
0,BeastyFeast,40.740583,-74.003878,basketball-court,"{'type': 'Point', 'coordinates': [40.740583, -..."
1,The Salty Paw,40.707577,-74.001743,basketball-court,"{'type': 'Point', 'coordinates': [40.707577, -..."
2,Biscuits & Bath,40.736371,-73.996113,basketball-court,"{'type': 'Point', 'coordinates': [40.736371, -..."
3,Eva's Play Pups,40.722154,-73.959235,basketball-court,"{'type': 'Point', 'coordinates': [40.722154, -..."
4,Dog Room Club,40.785382,-73.951538,basketball-court,"{'type': 'Point', 'coordinates': [40.785382, -..."
...,...,...,...,...,...
45,Starbucks,51.530942,-0.122104,starbucks,"{'type': 'Point', 'coordinates': [51.530942, -..."
46,Starbucks,51.516394,-0.103293,starbucks,"{'type': 'Point', 'coordinates': [51.516394, -..."
47,Starbucks,51.514326,-0.152723,starbucks,"{'type': 'Point', 'coordinates': [51.514326, -..."
48,Starbucks,51.514575,-0.143871,starbucks,"{'type': 'Point', 'coordinates': [51.514575, -..."


In [192]:
team_preferences_df.to_json("json/team_preferences.json", orient="records")

team_preferences_df.to_csv('data/all_team_preferences.csv')

top_cities.to_csv('data/top_cities_final.csv')

In [193]:
sf_all.to_csv('data/sf_all.csv')